<a href="https://colab.research.google.com/github/Pratiksha377/Breast-Cancer-Prediction-/blob/main/Copy_of_Untitled7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchtext==0.17.1
!pip install sacrebleu


In [ ]:
!pip install numpy==1.26.4 --force-reinstall --quiet
import os
os.kill(os.getpid(), 9)  # Restart the runtime


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
torchtune 0.6.1 requires torchdata==0.11.0, but you have torchdata 0.7.1 which is incompatible.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 2.2.1 which is incompatible.


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
import sacrebleu


In [ ]:
df = pd.read_csv('/content/pratikshasarma40.tsv', sep='\t')  # Replace with your CSV path
df = df[['English_Translation', 'Assamese_Sentence']].dropna()
display(df.head())


,English_Translation,Assamese_Sentence
0,"Moreover, Naba Kumar Sarania has expressed the...",তদুপৰি এইবেলিও পূৰ্বৰ দৰেই তেওঁ বিপুল ভোটত জয়ী...
1,Kuladhar Saikia was an IPS officer of 1985 batch.,১৯৮৫ বেটছৰ আই পি এছ বিষয়া আছিল কুলধৰ শইকীয়া।
2,"Like every year, this time too, the 55th Devi ...",প্ৰতিবছৰৰ দৰে এইবাৰো বহাগ মাহৰ কৃষ্ণ পক্ষৰ প্ৰ...
3,The moment he came to know about the failure f...,ইছৰ’ৰ অধ্যক্ষৰ পৰা এই বিজুতিৰ বিষয়ে অৱগত হোৱাৰ...
4,This is how the laddu is preparing for the Lok...,বৃহস্পতিবাৰে ঘোষণা হ’ব লগা লোকসভা নিৰ্বাচনৰ ফল...


In [ ]:
def tokenize_en(sentence):
    return sentence.lower().strip().split()

def tokenize_as(sentence):
    return sentence.strip().split()


In [ ]:
def yield_tokens(data, tokenizer):
    for sentence in data:
        yield tokenizer(sentence)

SRC_VOCAB = build_vocab_from_iterator(yield_tokens(df['English_Translation'], tokenize_en), specials=["<pad>", "<sos>", "<eos>", "<unk>"])
TGT_VOCAB = build_vocab_from_iterator(yield_tokens(df['Assamese_Sentence'], tokenize_as), specials=["<pad>", "<sos>", "<eos>", "<unk>"])

SRC_VOCAB.set_default_index(SRC_VOCAB["<unk>"])
TGT_VOCAB.set_default_index(TGT_VOCAB["<unk>"])
SRC_itos = SRC_VOCAB.get_itos()
TGT_itos = TGT_VOCAB.get_itos()

SRC_stoi = SRC_VOCAB.get_stoi()
TGT_stoi = TGT_VOCAB.get_stoi()


NameError: name 'build_vocab_from_iterator' is not defined

In [ ]:
def process_pair(src, tgt):
    src_tensor = [SRC_VOCAB["<sos>"]] + [SRC_VOCAB[token] for token in tokenize_en(src)] + [SRC_VOCAB["<eos>"]]
    tgt_tensor = [TGT_VOCAB["<sos>"]] + [TGT_VOCAB[token] for token in tokenize_as(tgt)] + [TGT_VOCAB["<eos>"]]
    return torch.tensor(src_tensor), torch.tensor(tgt_tensor)

pairs = [process_pair(en, asam) for en, asam in zip(df['English_Translation'], df['Assamese_Sentence'])]
train_data, test_data = train_test_split(pairs, test_size=0.2, random_state=42)


In [ ]:
def collate_fn(batch):
    src_batch, tgt_batch = zip(*batch)
    src_batch = pad_sequence(src_batch, padding_value=SRC_VOCAB["<pad>"])
    tgt_batch = pad_sequence(tgt_batch, padding_value=TGT_VOCAB["<pad>"])
    return src_batch.transpose(0, 1), tgt_batch.transpose(0, 1)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False, collate_fn=collate_fn)


In [ ]:
class Attention(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()
        self.attn = nn.Linear(hid_dim * 2, hid_dim)
        self.v = nn.Linear(hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        hidden = hidden.unsqueeze(1).repeat(1, encoder_outputs.shape[1], 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        attention = self.v(energy).squeeze(2)
        return torch.softmax(attention, dim=1)

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, batch_first=True)

    def forward(self, src):
        embedded = self.embedding(src)
        outputs, (hidden, cell) = self.lstm(embedded)
        return outputs, hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, attention):
        super().__init__()
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.attention = attention
        self.lstm = nn.LSTM(emb_dim + hid_dim, hid_dim, batch_first=True)
        self.fc = nn.Linear(hid_dim * 2, output_dim)

    def forward(self, input, hidden, cell, encoder_outputs):
        input = input.unsqueeze(1)
        embedded = self.embedding(input)
        attn_weights = self.attention(hidden[-1], encoder_outputs).unsqueeze(1)
        context = torch.bmm(attn_weights, encoder_outputs)
        rnn_input = torch.cat((embedded, context), dim=2)
        output, (hidden, cell) = self.lstm(rnn_input, (hidden, cell))
        prediction = self.fc(torch.cat((output.squeeze(1), context.squeeze(1)), dim=1))
        return prediction, hidden, cell


In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        batch_size, tgt_len = tgt.shape
        tgt_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(src.device)

        encoder_outputs, hidden, cell = self.encoder(src)
        input = tgt[:, 0]

        for t in range(1, tgt_len):
            output, hidden, cell = self.decoder(input, hidden, cell, encoder_outputs)
            outputs[:, t] = output
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            input = tgt[:, t] if teacher_force else output.argmax(1)

        return outputs


In [ ]:
NUM_EPOCHS = 38

INPUT_DIM = len(SRC_VOCAB)
OUTPUT_DIM = len(TGT_VOCAB)
EMB_DIM = 128
HID_DIM = 256

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

attn = Attention(HID_DIM)
enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, attn)
model = Seq2Seq(enc, dec).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=TGT_VOCAB['<pad>'])

print("🔁 Starting training for 60 epochs...\n")
for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    epoch_loss = 0

    for src, tgt in train_loader:
        src, tgt = src.to(device), tgt.to(device)
        optimizer.zero_grad()
        output = model(src, tgt)
        output = output[:, 1:].reshape(-1, output.shape[-1])
        tgt = tgt[:, 1:].reshape(-1)
        loss = criterion(output, tgt)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader)
    print(f"🟢 Epoch {epoch}/{NUM_EPOCHS} | Loss: {avg_loss:.4f}")

# Optional: Save after training
torch.save(model.state_dict(), "eng_to_asam_lstm_attn_60epoch.pth")
print("\n✅ Model saved as 'eng_to_asam_lstm_attn_60epoch.pth'")

from IPython.display import FileLink
FileLink("eng_to_asam_lstm_attn_60epoch.pth")



🔁 Starting training for 60 epochs...

🟢 Epoch 1/38 | Loss: 8.7618
🟢 Epoch 2/38 | Loss: 8.0088
🟢 Epoch 3/38 | Loss: 7.2276
🟢 Epoch 4/38 | Loss: 6.0901
🟢 Epoch 5/38 | Loss: 4.8157
🟢 Epoch 6/38 | Loss: 3.8211
🟢 Epoch 7/38 | Loss: 3.3152
🟢 Epoch 8/38 | Loss: 2.9142
🟢 Epoch 9/38 | Loss: 2.5354
🟢 Epoch 10/38 | Loss: 2.1948
🟢 Epoch 11/38 | Loss: 1.8581
🟢 Epoch 12/38 | Loss: 1.6100
🟢 Epoch 13/38 | Loss: 1.3575
🟢 Epoch 14/38 | Loss: 1.1417
🟢 Epoch 15/38 | Loss: 0.9247
🟢 Epoch 16/38 | Loss: 0.7470
🟢 Epoch 17/38 | Loss: 0.5963
🟢 Epoch 18/38 | Loss: 0.4670
🟢 Epoch 19/38 | Loss: 0.3678
🟢 Epoch 20/38 | Loss: 0.2878
🟢 Epoch 21/38 | Loss: 0.2337
🟢 Epoch 22/38 | Loss: 0.1909
🟢 Epoch 23/38 | Loss: 0.1541
🟢 Epoch 24/38 | Loss: 0.1277
🟢 Epoch 25/38 | Loss: 0.1195
🟢 Epoch 26/38 | Loss: 0.1121
🟢 Epoch 27/38 | Loss: 0.1066
🟢 Epoch 28/38 | Loss: 0.0951
🟢 Epoch 29/38 | Loss: 0.0986
🟢 Epoch 30/38 | Loss: 0.0982
🟢 Epoch 31/38 | Loss: 0.0888
🟢 Epoch 32/38 | Loss: 0.0772
🟢 Epoch 33/38 | Loss: 0.0673
🟢 Epoch 34/38 

/content/eng_to_asam_lstm_attn_60epoch.pth

In [ ]:
import torch
torch.save(model.state_dict(), "eng_to_asam_lstm_attn_60epoch.pth")
print("✅ Model weights saved!")


✅ Model weights saved!


In [ ]:
from google.colab import files
files.download("eng_to_asam_lstm_attn_60epoch.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.5 MB/s eta 0:00:00


In [ ]:
from sacrebleu.metrics import BLEU

INPUT_DIM = len(SRC_VOCAB)
OUTPUT_DIM = len(TGT_VOCAB)
EMB_DIM = 128
HID_DIM = 256

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

attn = Attention(HID_DIM)
enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, attn)

model = Seq2Seq(enc, dec).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=TGT_VOCAB['<pad>'])

# 🔁 Train for One Epoch
model.train()
for src, tgt in train_loader:
    src, tgt = src.to(device), tgt.to(device)
    optimizer.zero_grad()
    output = model(src, tgt)
    output = output[:, 1:].reshape(-1, output.shape[-1])
    tgt = tgt[:, 1:].reshape(-1)
    loss = criterion(output, tgt)
    loss.backward()
    optimizer.step()

print(f"\n✅ Training Complete! Final Loss: {loss.item():.4f}\n")

# 🔎 Evaluation with Sentence-wise and Overall BLEU
model.eval()
bleu = BLEU()
pred_sentences = []
true_sentences = []
sentence_bleus = []

with torch.no_grad():
    for src, tgt in test_loader:
        src = src.to(device)
        encoder_outputs, hidden, cell = model.encoder(src)
        input = torch.tensor([TGT_VOCAB['<sos>']] * src.size(0)).to(device)

        preds = []
        for _ in range(30):
            output, hidden, cell = model.decoder(input, hidden, cell, encoder_outputs)
            top1 = output.argmax(1)
            preds.append(top1.item())
            input = top1
            if top1.item() == TGT_VOCAB['<eos>']:
                break

        pred_tokens = [TGT_VOCAB.get_itos()[idx] for idx in preds if idx not in [TGT_VOCAB['<pad>'], TGT_VOCAB['<sos>'], TGT_VOCAB['<eos>']]]
        true_tokens = [TGT_VOCAB.get_itos()[idx.item()] for idx in tgt[0] if idx.item() not in [TGT_VOCAB['<pad>'], TGT_VOCAB['<sos>'], TGT_VOCAB['<eos>']]]

        pred_sentence = " ".join(pred_tokens)
        true_sentence = " ".join(true_tokens)

        pred_sentences.append(pred_sentence)
        true_sentences.append(true_sentence)

        sentence_bleu = bleu.sentence_score(pred_sentence, [true_sentence]).score
        sentence_bleus.append(sentence_bleu)

        print(f"🔵 Ground Truth   : {true_sentence}")
        print(f"🟢 Prediction     : {pred_sentence}")
        print(f"🎯 Sentence BLEU  : {sentence_bleu:.2f}\n")

# 🌐 Corpus BLEU
corpus_bleu = bleu.corpus_score(pred_sentences, [true_sentences])
print("🌟 Model BLEU Score:", f"{corpus_bleu.score:.2f}")


NameError: name 'SRC_VOCAB' is not defined

In [ ]:
# Re-initialize architecture exactly like before
attn = Attention(HID_DIM)
enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, attn)
model = Seq2Seq(enc, dec).to(device)

# Load saved weights
model.load_state_dict(torch.load("eng_to_asam_lstm_attn_60epoch.pth"))
model.eval()


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(23757, 128)
    (lstm): LSTM(128, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(34343, 128)
    (attention): Attention(
      (attn): Linear(in_features=512, out_features=256, bias=True)
      (v): Linear(in_features=256, out_features=1, bias=False)
    )
    (lstm): LSTM(384, 256, batch_first=True)
    (fc): Linear(in_features=512, out_features=34343, bias=True)
  )
)

In [ ]:
SRC_VOCAB = build_vocab(df['src_tokens'])
TGT_VOCAB = build_vocab(df['tgt_tokens'])

SRC_itos = {i: s for s, i in SRC_VOCAB.items()}
TGT_itos = {i: s for s, i in TGT_VOCAB.items()}


NameError: name 'build_vocab' is not defined

In [ ]:
def translate_sentence(model, sentence, src_vocab, tgt_vocab, tgt_itos, max_len=50):
    model.eval()

    # Tokenize and convert to indices
    tokens = sentence.lower().strip().split()
    src_ids = [src_vocab.get(token, src_vocab['<unk>']) for token in tokens]
    src_tensor = torch.tensor(src_ids).unsqueeze(0).to(device)

    with torch.no_grad():
        encoder_outputs, hidden, cell = model.encoder(src_tensor)
        input_token = torch.tensor([tgt_vocab['<sos>']]).to(device)
        result_ids = []

        for _ in range(max_len):
            output, hidden, cell = model.decoder(input_token, hidden, cell, encoder_outputs)
            top1 = output.argmax(1).item()
            if top1 == tgt_vocab['<eos>']:
                break
            result_ids.append(top1)
            input_token = torch.tensor([top1]).to(device)

    translated = [tgt_itos[idx] for idx in result_ids]
    return ' '.join(translated)

# 🔄 Example usage:
while True:
    input_text = input("🔤 Enter an English sentence (or type 'exit' to quit): ")
    if input_text.lower() == 'exit':
        break
    translation = translate_sentence(model, input_text, SRC_stoi, TGT_stoi, TGT_itos)
    print("🌐 Assamese:", translation)


🔤 Enter an English sentence (or type 'exit' to quit): Hello
🌐 Assamese: স্কঃ কলিয়াবৰত এটা দিনত দিয়াৰ লগতে সত্ৰীয়া বশিষ্ঠ দেৱ গৰাকীৰ বিৰুদ্ধে শূন্য কৰি
🔤 Enter an English sentence (or type 'exit' to quit): My name is
🌐 Assamese: মোৰ ৰাম নাথ কোবিন্দৰ পৰা মোৰ সাধাৰণ সম্পাদক লুৰিণজ্যোতি গগৈয়ে মোৰ শিৰ দোঁ খাই
🔤 Enter an English sentence (or type 'exit' to quit): My name is Pratiksha
🌐 Assamese: মোৰ ৰাম নাথ মোৰ তেওঁলৈ প্ৰৱেশত দি এনেদৰে মই মোৰ উপায়ুক্ত আৰু


KeyboardInterrupt: Interrupted by user